In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import torch
import os
from torch_geometric.datasets import PPI
from torch_geometric.utils import to_dense_adj
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha,head, concat=True):
        super(GATLayer, self).__init__()
        self.dropout       = dropout
        self.in_features   = in_features
        self.out_features  = out_features
        self.alpha         = alpha
        self.concat        = concat
        self.head = head
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(self.head,2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, edge_index):
        h = torch.mm(input, self.W)
        N = h.size()[0]
        adj1 = to_dense_adj(edge_index)
        adj1=adj1[0]
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = torch.zeros(self.head,2708,2708)
        for i in range(self.head):
          e[i] = self.leakyrelu(torch.matmul(a_input, self.a[i]).squeeze(2))
        # print(e.shape)
        # print("u")
        zero_vec = -9e15 * torch.ones_like(e)
        attention  = torch.zeros_like(e)
        # print(attention.shape)
        for i  in range(self.head):
          attention[i] = torch.where(adj1 > 0, e[i], zero_vec[i])
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        attentiont = torch.zeros_like(attention[0])
        for i in range (self.head):
          attentiont+=attention[i]
        attentiont/=self.head
        # print("l")
        # print(self.W.shape)
        # print(attentiont.shape)
        # print(h.shape)
        h_prime = torch.matmul(attentiont, h)
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

class GAT(torch.nn.Module):
    def __init__(self, dataset):
        super(GAT, self).__init__()
        self.hid = 8
        self.conv1 = GATLayer(dataset.num_node_features, self.hid, alpha=0.2,head =8, dropout=0.6)
        self.conv2 = GATLayer(self.hid, dataset.num_classes, concat=False, alpha=0.2,head =1, dropout=0.6)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T


# Load the CORA dataset
dataset = Planetoid(root='./data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]



In [ ]:
print(dataset.num_classes)

7


In [ ]:
labels = data.y.cpu().numpy()
label_to_indices = {}
for i in range(dataset.num_classes):
    label_to_indices[i] = (labels == i).nonzero()[0]
import numpy as np
train_indices = []
val_indices = []
test_indices = []
np.random.seed(42)

for label, indices in label_to_indices.items():
    np.random.shuffle(indices)
    train_indices.extend(indices[:20])
    val_indices.extend(indices[20:20+500])
    test_indices.extend(indices[20+500:20+500+1000])
import torch

train_mask = torch.zeros_like(data.y, dtype=torch.bool)
train_mask[train_indices] = True

val_mask = torch.zeros_like(data.y, dtype=torch.bool)
val_mask[val_indices] = True

test_mask = torch.zeros_like(data.y, dtype=torch.bool)
test_mask[test_indices] = True
from torch_geometric.data import Data
data = Data(x=data.x, edge_index=data.edge_index, y=data.y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
model = GAT(dataset)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.0005)
criterion = nn.NLLLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    # print(out.shape)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def validate():
    model.eval()
    raw = model(data)
    accs = []
    for mask in [train_mask, val_mask, test_mask]:
        pred = raw[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for e in range(1, 201):
    l = train()
    t, v, te = validate()
    print(f'Epoch: {e}, Loss: {l:.4f}, Train: {t:.4f}, Val: {v:.4f}, Test: {te:.4f}')


Epoch: 001, Loss: 1.9477, Train: 0.2786, Val: 0.1780, Test: 0.2349
Epoch: 002, Loss: 1.9443, Train: 0.4000, Val: 0.2881, Test: 0.3490
Epoch: 003, Loss: 1.9363, Train: 0.6286, Val: 0.4150, Test: 0.4497
Epoch: 004, Loss: 1.9351, Train: 0.7286, Val: 0.5207, Test: 0.5235
Epoch: 005, Loss: 1.9265, Train: 0.7857, Val: 0.5714, Test: 0.5772
Epoch: 006, Loss: 1.9215, Train: 0.8143, Val: 0.6167, Test: 0.5872
Epoch: 007, Loss: 1.9154, Train: 0.8429, Val: 0.6502, Test: 0.6107
Epoch: 008, Loss: 1.9134, Train: 0.8500, Val: 0.6696, Test: 0.6376
Epoch: 009, Loss: 1.9086, Train: 0.8571, Val: 0.6863, Test: 0.6510
Epoch: 010, Loss: 1.8979, Train: 0.8786, Val: 0.6960, Test: 0.6510
Epoch: 011, Loss: 1.8955, Train: 0.9000, Val: 0.7048, Test: 0.6745
Epoch: 012, Loss: 1.8936, Train: 0.9143, Val: 0.7093, Test: 0.6846
Epoch: 013, Loss: 1.8935, Train: 0.9143, Val: 0.7141, Test: 0.7081
Epoch: 014, Loss: 1.8640, Train: 0.9286, Val: 0.7203, Test: 0.7114
Epoch: 015, Loss: 1.8718, Train: 0.9214, Val: 0.7251, Test: 0.

In [ ]:
|print(data.x.shape)

torch.Size([2708, 1433])


In [ ]:
pip install torch-geometric